Imports

In [1]:
from plantfusion.lgrass_wrapper import Simpraise_wrapper # L-grass FSPM management : TO BE IMPLEMENTED
# see : https://github.com/mwoussen/plantfusion/blob/develop/simulations/other_fspm.py
from plantfusion.light_wrapper import Light_wrapper # light management using LightVegeManager
from plantfusion.soil_wrapper import Soil_wrapper # soil management
from plantfusion.indexer import Indexer # index management
from plantfusion.planter import Planter # plant position management

import time
import datetime
import os

Path management

In [2]:
in_folder_lgrass = "inputs_lgrass/inputs"
genet_folder_lgrass = "inputs_lgrass/modelgenet"
out_folder = "outputs/lgrass"

id1 = 17111
id2 = 17112
writegeo = True

try:
    # Create target Directory
    os.mkdir(os.path.normpath(out_folder))
    print("Directory ", os.path.normpath(out_folder), " Created ")
except FileExistsError:
    print("Directory ", os.path.normpath(out_folder), " already exists")


Directory  outputs\lgrass  already exists


Already implemented in plantfusion (should run if given appropriate parameters)

In [3]:
lgrass_name = "lgrass"

indexer = Indexer(global_order=[lgrass_name], other_names=[lgrass_name])

generation_type = "random"
plant_density = {lgrass_name : 450} # plantes.m-2
planter = Planter(generation_type=generation_type, indexer=indexer, plant_density=plant_density)
#note : arg "inter_rows" is not needed for random generation (I think), but is set to 0.15 anyway.

sky = "turtle46"
lighting_caribu = Light_wrapper(
        lightmodel="caribu",
        indexer=indexer, 
        planter=planter, 
        sky=sky,
        writegeo=False,
    )


soil = Soil_wrapper(
    in_folder='inputs_soil_legume', 
    out_folder=out_folder, 
    IDusm=1711, # unit of simulation
    planter=planter, 
    opt_residu=0, 
    save_results=True
)


Directory  outputs\lgrass\soil  already exists


Lgrass Wrapper instanciation

In [4]:
lgrass = Simpraise_wrapper(
    name=lgrass_name,
    indexer=indexer,
    in_folder=in_folder_lgrass, genet_folder=genet_folder_lgrass, out_folder=out_folder,
    planter=planter,
    plan_sim_file='plan_simulation.csv',
    genet_file='ped.r',
    param_plant_file='liste_plantes.csv'
    #ADD ARGS SPECIFIC TO L-GRASS
)

<string>:1402: Warning: Found symbol '#' after Lstring. Considered as begining of comments
lgrass.lpy:1599: Warning: IndentationWarning: missing indent.
lgrass.lpy:1614: Warning: IndentationWarning: missing indent.
lgrass.lpy:1730: Warning: IndentationWarning: missing indent.
c:\users\u242825\desktop\gembloux\recherche\plant_modelling\lgrass_legume_fusion\plantfusion\lgrass\lgrass\param_reproduction_functions.py:140: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform

Simulation Lgrass alone

In [5]:
try:
    current_time_of_the_system = time.time()
    for t in range(lgrass.lsystem.derivationLength):
        
        lgrass.derive(t)

        ### CARIBU
        print(lgrass.lstring)
        scene_lgrass = lgrass.light_inputs(elements="triangles")
        

        if len(scene_lgrass) > 0:
            start = time.time()
            lighting_caribu.run(
                scenes=[scene_lgrass], day=lgrass.lsystem.current_day, parunit="RG"
            )
            caribu_time = time.time() - start
            lgrass.light_results()

            lgrass.run()

            print("Lighting running time | CARIBU: ", caribu_time)

    execution_time = int(time.time() - current_time_of_the_system)
    print("\n" "Simulation run in {}".format(str(datetime.timedelta(seconds=execution_time))))

finally:
    lgrass.end()

TPS : 1.0 Current day : 1
CouvertVegetal(1)
TPS : 2.0 Current day : 1
CouvertVegetal(2)IN[\(45)@M(0.0,0.0,0)Plante(<lgrass.ParamPlante object at 0x0000029CF3B56B80>)IN]IN[\(45)@M(50.0,0.0,0)Plante(<lgrass.ParamPlante object at 0x0000029CF3B56430>)IN]IN[\(45)@M(0.0,50.0,0)Plante(<lgrass.ParamPlante object at 0x0000029CF3B56310>)IN]IN[\(45)@M(50.0,50.0,0)Plante(<lgrass.ParamPlante object at 0x0000029CF3B56D60>)IN]
TPS : 3.0 Current day : 1
id_plante :  0
id_talle :  0
id_plante :  1
id_talle :  0
id_plante :  2
id_talle :  0
id_plante :  3
id_talle :  0
CouvertVegetal(3)IN[\(45)@M(0.0,0.0,0)Plante(<lgrass.ParamPlante object at 0x0000029CF3B56B80>)phytomere(<lgrass.ParamPhytomere object at 0x0000029CF3855C40>,<lgrass.ParamApex object at 0x0000029CF3B56F70>)\(95)bourgeonRoot(<lgrass.ParambourgeonRoot object at 0x0000029CF3B56F40>,<lgrass.ParamEntrenoeud object at 0x0000029CF38D3D90>)ApexTal(<lgrass.ParamApexTal object at 0x0000029CF3B563A0>)Entrenoeud(<lgrass.ParamEntrenoeud object at 0x00

NameError: Il n'y a pas eu suffisamment de graines produites pour établir une nouvelle génération.